# Exploratory Analysis

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from olist.data import Olist
data = Olist().get_data()

### 1 - Run an exploratory analysis with [pandas profiling](https://github.com/pandas-profiling/pandas-profiling)

In [0]:
# First, let's install the pandas-profiling package
! pip install pandas-profiling

In [0]:
# And create a new "04-Decision-Science/reports" folder 
!mkdir -p ../../data/reports

In [0]:
import pandas_profiling
datasets_to_profile = ['orders', 'products', 'sellers',
                  'customers', 'order_reviews',
                  'order_items']

👉 Create and save one `html report` per dataset to profile 

⏳ (It usually takes a few minutes)

In [0]:
for d in datasets_to_profile:
    print('exporting: '+d)
    profile = data[d].profile_report(title='Report for '+d)
    profile.to_file(output_file="../../data/reports/"+d+'.html');

Take 10 min to read the reports 📈📊, and feel free to add insights of your choice to your `db.lewagon.org schema`

### 2 - Create a `matching table` (a.k.a a `star_table`)

[**`The STAR Schema`**](https://en.wikipedia.org/wiki/Star_schema )

<img src="https://docs.microsoft.com/en-us/power-bi/guidance/media/star-schema/star-schema-example1.png" width=700>

_From Wikipedia:_


> The star schema gets its name from the physical model's resemblance to a star shape with a fact table at its center and the dimension tables surrounding it representing the star's points.

> The star schema separates business process data into facts, which hold the measurable, quantitative data about a business, and dimensions which are descriptive attributes related to fact data. 

_Application to Olist_

A **`fact`** on the Olist ecommerce platform is characterized by:
- a `customer`, that would buy
- a `product`
- to a `seller`
- and leaves a `rewiew`
- belonging to an `order_id`

☝️ It contains the most important foreign keys together, for later use during the module.

**Dimension** tables may include 
- `products` details table: (models colors, sizes)
- `sellers` details table (geographic locations...)
- `customers` details table (age, name, ...)
- `reviews` details table (raw text, review_date,...)
- `orders` details table (payment id, payment amount,...)

☝️ We will create these Dimension tables later

❓ Let's create the Fact Table that we will call `matching_table`

In [0]:
# matching_table is a DataFrame with the following columns (below).  
columns_matching_table = [
    "order_id",
    "review_id",
    "customer_id",
    "product_id",
    "seller_id",
]

👉 **(Follow our guide: 1/3)** To create this `matching_table`, select carefully the columns of interest in some Olist datasets. Inspect the cardinality of each DataFrame using `pd.DataFrame.shape` and `pd.Series.nunique()`

In [0]:
# Select only the columns of interest in the various dataframes of interest, before proceeding to any merge
orders = None #data[??]
reviews = None #data[??]
items = None #data[??]

In [0]:
# Select only the columns of interest in the various dataframes of interest, before proceeding to any merge
orders = data['orders'][['customer_id', 'order_id']]
reviews = data['order_reviews'][['order_id', 'review_id']]
items = data['order_items'][['order_id', 'product_id','seller_id']]

In [0]:
print('orders:', orders.shape, orders.customer_id.nunique(), 'unique customer_ids, and', orders.order_id.nunique(), 'unique order_ids')
print('review: ', reviews.shape, reviews.order_id.nunique(), 'unique order_ids and', reviews.review_id.nunique(), 'unique reviews' )
print('items: ', items.shape, items.order_id.nunique(), 'unique order_ids,', items.product_id.nunique(), 
      'unique product_ids, and', items.seller_id.nunique(), 'unique seller_ids')

orders: (99441, 2) 99441 unique customer_ids, and 99441 unique order_ids
review:  (99224, 2) 98673 unique order_ids and 98410 unique reviews
items:  (112650, 3) 98666 unique order_ids, 32951 unique product_ids, and 3095 unique seller_ids


👉 **(2/3)**:  Merge these Dataframes carefully to create the `matching_table`

_Note: Use outer joins to make sure you don't lose any information at this stage_

In [0]:
# Carefully merge DataFrames
matching_table = orders.merge(reviews, on='order_id', how='outer').merge(items, on='order_id', how='outer')
matching_table

,customer_id,order_id,review_id,product_id,seller_id
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,a54f0611adc9ed256b57ede6b6eb5114,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,8d5266042046a06655c8db133d120ba5,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,e73b67b67587f7644d5bd1a52deb1b01,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,359d03e676b3c069f62cadba8dd3f6e8,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,e50934924e227544ba8246aeb3770dd4,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8
...,...,...,...,...,...
114087,1fca14ff2861355f6e5f14306ff977a7,63943bddc261676b46f01ca7ac2f7bd8,29bb71b2760d0f876dfa178a76bc4734,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554
114088,1aa71eb042121263aafbe80c1b562c9c,83c1379a015df1e13d02aae0204711ab,371579771219f6db2d830d50805977bb,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48
114089,b331b74b18dc79bcdf6532d51e1637c1,11c177c8e97725db2631073c19f07b62,8ab6855b9fe9b812cd03a480a25058a1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48
114090,b331b74b18dc79bcdf6532d51e1637c1,11c177c8e97725db2631073c19f07b62,8ab6855b9fe9b812cd03a480a25058a1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48


In [0]:
matching_table.isna().sum()

customer_id      0
order_id         0
review_id      961
product_id     778
seller_id      778
dtype: int64

👉 **(3/3):** Does this `matching_table` have duplicated rows ?  
- How many duplicates ?
- If so, what could be the reason(s) ?

<details>
    <summary>▸ <i>Hints</i></summary> 
    
* For a given `order_id`, the quantity of a given `product_id` bought can be greater than 1
* In the `items` table, each individual product bought appears as an additional row. 

</details>

In [0]:
matching_table.duplicated().sum()

10287

☝️ So, we have duplicated rows because in a given fact, a product could have been ordered multilple times!

❓ Group your matching table so as to have zero dupicates, but a `number_of_products` column instead

In [0]:
matching_table["number_of_products"] = 1

In [0]:
list(matching_table.columns)

['customer_id',
 'order_id',
 'review_id',
 'product_id',
 'seller_id',
 'number_of_products']

In [0]:
# Groupby and count non-NA cells for each column.
matching_table = matching_table.groupby(['customer_id',
                                         'order_id',
                                         'review_id',
                                         'product_id',
                                         'seller_id'],
                                        as_index = False)\
            .count()\
            .sort_values(by = "number_of_products", ascending = False)

matching_table

,customer_id,order_id,review_id,product_id,seller_id,number_of_products
76202,be1b70680b9f9694d8c70f41fa3dc92b,1b15974a0141d54e36626dca3fdc731a,be332150a9c96e68c9565ea53cba2355,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,20
6697,10de381f8a8d23fff822753305f71cae,428a2f660dc84138d969ccd69a0ab6d5,d638a70f2be180ef55395eabb78fd88c,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,15
69494,adb32467ecc74b53576d9d13a5a55891,9ef13efd6949e4573a18964dd1bbe7f5,2e3a6e4930334530774ac3a6f6b62388,37eb69aca8718e843d897aa7b82f462d,0b36063d5818f81ccb94b54adfaebbf5,15
85719,d5f2b3f597c7ccafbb5cac0bcc3d6024,73c8ab38f07dc94389065f7eba4f297a,ee4bc8e340e8648a44c2e33fee6b27e4,422879e10f46682990de24d770e7f83d,1f50f920176fa81dab994f9023523100,14
66856,a7693fba2ff9583c78751f2b66ecab9d,9bdc4d4c71aa1de4606060929dee888c,03129dea7c12fa5878b2e629ccdf2ce6,44a5d24dd383324a421569ca697b13c2,e7d5b006eb624f13074497221eb37807,14
...,...,...,...,...,...,...
35329,58e73c82bbd5992b6ca53e1dcd0aaf16,79c786a94283e846deec8681219bc144,98b4f06a4499ec71eddaa09c33a08881,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,1
35328,58e6ef977832bfd6f5c99001fc47432b,b7f12e29e55ecae289d444db518400ed,06586453b01b4b23beaea71738457789,e3fa14bf3866f013d03fa0b76a9842ee,8a32e327fe2c1b3511609d81aaf9f042,1
35327,58e67afe5b8a9fa07e268e00776afbdc,277006d8d593850cce4c18ce1c9a136a,c05178d7b277bc0f85b83f2f70e2bc94,1613b819ab5dae53aead2dbb4ebdb378,16090f2ca825584b5a147ab24aa30c86,1
35325,58e41ee664edb6b56cfa39832d76d01e,e300c76076c4b56031abe6f3ff54817f,b186de4f017163562b6015e68e0c778d,9eae1df2c1112be7a7c57d7b09fd9bcd,0bebbb2cea103a4a020c95d43fd7d754,1


In [0]:
matching_table.isna().sum()

customer_id           0
order_id              0
review_id             0
product_id            0
seller_id             0
number_of_products    0
dtype: int64

👉 Inspect the shape and the number of uniques values in  of the final DataFrame - *Hint*: use `nunique()`

🎯 It should match (102230, 6) and contains no NaN

In [0]:
print(f"matching_table shape = {matching_table.shape}")
print("-"*50)
print('unique values: ')
print(matching_table.nunique())

matching_table shape = (102230, 6)
--------------------------------------------------
unique values: 
customer_id           97917
order_id              97917
review_id             97709
product_id            32789
seller_id              3090
number_of_products       16
dtype: int64


### 3 - Save your logic in `data.py` 

❓Copy your logic into `get_matching_table()` in `data.py` and run the cell below to check your code!

In [0]:
from nbresult import ChallengeResult
from olist.data import Olist

data = Olist().get_matching_table()

result = ChallengeResult('matching_table',
    shape=data.shape,
    columns=sorted(list(data.columns)) 
)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/brunolajoie/.pyenv/versions/lewagon3812/bin/python3
cachedir: .pytest_cache
rootdir: /Users/brunolajoie/code/lewagon/data-solutions/04-Decision-Science/01-Project-Setup/03-Exploratory-Analysis
plugins: anyio-3.3.0, dash-2.0.0
collecting ... collected 2 items

tests/test_matching_table.py::TestMatchingTable::test_columns PASSED     [ 50%]
tests/test_matching_table.py::TestMatchingTable::test_shape PASSED       [100%]

============================== 2 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/matching_table.pickle

git commit -m 'Completed matching_table step'

git push origin master
